Selenium web driver is availible [here](https://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_mac64.zip)

In [1]:
import csv
import json
import time

from urllib.request import urlopen
from bs4 import BeautifulSoup, element
from bidi.algorithm import get_display
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
webdriver_path = r'/Users/evgenys/GIT/kfar_scrape/chromedriver'
URL='https://www.carmella.co.il/product-category/%d7%99%d7%a8%d7%a7%d7%95%d7%aa/'
DATA_PATH = '/Users/evgenys/GIT/kfar_scrape/DATA'
CSV_FIELD_NAMES = [
    'Product_ID',
    'Product_Name',
    'Description',
    'Price',
    'Price_Type',
    'New_Product',
    'Sale',
    'Image_URL']
no_of_pagedowns = 50

In [3]:
# Path to webdriver
browser = webdriver.Chrome(webdriver_path)

# URL to scrape
browser.get(URL)
time.sleep(1)

elem = browser.find_element_by_tag_name("body")

### We need to press down key until we get all the JS defined products

In [4]:
while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    no_of_pagedowns-=1

### Starting HTML analyzing

In [39]:
soup = BeautifulSoup(browser.page_source,"html.parser")
all_products = soup.find_all('div', {"class": 'product_wrap'})

In [126]:
product = all_products[102]

In [127]:
product_id = product['data-product-id']
product_cat_name = product.find_parents('div', {"class": 'subcat_with_items'})[0]['name']
product_image_url = product.find_all('div', {"class": 'product_img'})[0].find_all('img')[0]['data-src']
product_block = product.find_all('div', {"class": 'prod_bottom'})[0]
product_name = product_block.find_all('h3', {"class": 'pr_title'})[0].contents[0]
product_price = product_block.find_all('span', {"class": 'pr_price'})[0].contents[0].strip()
product_price_type = product_block.find_all('span', {"class": 'pr_price_kilo'})[0].contents[0].replace('/', '').strip()

isSale = 0
isNew = 0
if len(product_block.find_all('span', {"class": 'pr_title_sale'})) or len(product_block.find_all('span', {"class": 'pr_price_old'})): 
    isSale = 1

product_name_extra = ''
if len(product_block.find_all('span', {"class": 'pr_title_note'})):
    product_name_extra = product_block.find_all('span', {"class": 'pr_title_note'})[0].contents[0]

In [128]:
row = {
    'Product_ID': product_id,
    'Product_Name': product_name,
    'Description': product_name_extra,
    'Price': product_price,
    'Price_Type': product_price_type,
    'New_Product': isNew,
    'Sale': isSale,
    'Image_URL': product_image_url}

In [129]:
row

{'Product_ID': '52921',
 'Product_Name': 'שום שחור קלוף - Eravital',
 'Description': '125 גרם',
 'Price': '39.90',
 'Price_Type': 'יח׳',
 'New_Product': 0,
 'Sale': 1,
 'Image_URL': 'https://w9y3w7q3.stackpathcdn.com/wp-content/uploads/2020/02/8681644498969-300x255.jpg'}